In [3]:
import os
os.chdir('../')

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import folium

In [5]:
old_score = pd.read_csv("data/old_risco_imob.csv")[["id_imobiliaria", "risco_imobiliaria"]].query("risco_imobiliaria != 0")
df_imobs = pd.read_csv("notebooks/outputs/df_imobs.csv")[["id_imobiliaria", "lat_imob", "long_imob"]].drop_duplicates()

In [6]:
df_bins = pd.read_csv("notebooks/outputs/df_bins.csv")
df_bins.head()

,dt_ativacao,id_imobiliaria,predictions,target,old_segments,optimal_bins,optimal_segments,old_transformed_segments
0,2022-01,38,0.033534,0,B,0.001813,1-baixo,B
1,2022-01,50,0.033887,0,B,0.001813,1-baixo,B
2,2022-01,63,0.031284,0,A,0.001813,1-baixo,A
3,2022-01,81,0.135855,0,E,0.055794,3-alto,C
4,2022-01,95,0.097252,0,E,0.055794,3-alto,C


In [7]:
df_bins.groupby("optimal_segments")["id_imobiliaria"].nunique()

optimal_segments
1-baixo    353
2-medio    321
3-alto     173
Name: id_imobiliaria, dtype: int64

In [8]:
old_score["segments_old"] = (
    old_score["risco_imobiliaria"]
    .transform(
        lambda x: "A" if x == 2      else
                  "B" if x in (3, 4) else 
                  "C" if x == 5      else 
                  "D" if x == 6      else 
                  "E" if x == 7      else 
                  None
    )
)

In [9]:
old_score.groupby("segments_old").size()

segments_old
A     48
B      7
C    189
D    292
E    359
dtype: int64

In [10]:
df_loc = df_bins.merge(df_imobs, how="left", on="id_imobiliaria")

df_baixo = df_loc.query("optimal_segments == '1-baixo' and dt_ativacao == '2022-06'")
df_medio = df_loc.query("optimal_segments == '2-medio' and dt_ativacao == '2022-06'")
df_alto = df_loc.query("optimal_segments == '3-alto' and dt_ativacao == '2022-06'")

In [11]:
map = folium.Map(location=[-23.5558,-46.6396], zoom_start=6)


for lat, long in df_baixo[["lat_imob", "long_imob"]].itertuples(index=False):
    folium.Marker(
        location=[lat, long],
        popup="Baixo Risco",
        icon=folium.Icon(color="blue", icon="arrow-up"),
    ).add_to(map)

for lat, long in df_medio[["lat_imob", "long_imob"]].itertuples(index=False):
    folium.Marker(
        location=[lat, long],
        popup="Médio Risco",
        icon=folium.Icon(color="orange", icon="minus"),
    ).add_to(map)

for lat, long in df_alto[["lat_imob", "long_imob"]].itertuples(index=False):
    folium.Marker(
        location=[lat, long],
        popup="Alto Risco",
        icon=folium.Icon(color="red", icon="arrow-down"),
    ).add_to(map)

map